# Review and Rectify Loop

This notebook demonstrates a human-in-the-loop workflow for iteratively fixing content based on validation feedback.


## Setup


In [ ]:
# Install SDK
!pip install -e /content/dorc-clients/sdk/python


In [ ]:
import os
from datetime import datetime
from dorc_client import DorcClient

# Configure environment
os.environ["DORC_BASE_URL"] = "https://your-engine-url.run.app"
os.environ["DORC_TENANT_SLUG"] = "my-tenant"

# Create client
client = DorcClient()

# Track validation attempts
attempts = []


## Initial Candidate Content

Edit the `candidate_text` variable below with your initial content.


In [ ]:
# Initial candidate content
candidate_text = """# My Document

This is the initial content that may have contradictions.
"""

candidate_id = "iterative-test-001"

print(f"Candidate ID: {candidate_id}")
print(f"Content length: {len(candidate_text)} characters")


## Validation Loop

Run the cell below to validate. If failures are found, edit `candidate_text` above and re-run this cell.


In [ ]:
# Validate current content
print(f"\n{'='*60}")
print(f"Validation Attempt #{len(attempts) + 1}")
print(f"Timestamp: {datetime.now().isoformat()}")
print(f"{'='*60}\n")

response = client.validate(
    content=candidate_text,
    candidate_id=candidate_id,
)

# Store attempt
attempt = {
    "timestamp": datetime.now().isoformat(),
    "run_id": response.run_id,
    "summary": {
        "pass": response.content_summary.pass_,
        "fail": response.content_summary.fail,
        "warn": response.content_summary.warn,
        "error": response.content_summary.error,
    },
    "pipeline_status": response.pipeline_status
}
attempts.append(attempt)

# Display results
summary = response.content_summary
total = summary.pass_ + summary.fail + summary.warn + summary.error

print("Validation Results:")
print(f"  Run ID: {response.run_id}")
print(f"  ✅ PASS:  {summary.pass_:3d}/{total}")
print(f"  ⚠️  WARN:  {summary.warn:3d}/{total}")
print(f"  ❌ FAIL:  {summary.fail:3d}/{total}")
if summary.error > 0:
    print(f"  🔴 ERROR: {summary.error:3d}/{total}")

print(f"\n{'='*60}")
if summary.fail > 0:
    print("❌ VALIDATION FAILED")
    print("\nTo fix:")
    print("  1. Review the failed chunks below")
    print("  2. Edit the 'candidate_text' variable in the cell above")
    print("  3. Re-run this cell to validate again")
    print("\nFailed chunks:")
    for chunk in response.chunks:
        if chunk.status == "FAIL":
            print(f"\n  Chunk {chunk.index}: {chunk.message}")
            if chunk.evidence:
                for ev in chunk.evidence:
                    if ev.note:
                        print(f"    - {ev.note}")
elif summary.warn > 0:
    print("⚠️  VALIDATION PASSED WITH WARNINGS")
    print("\nWarnings found (non-blocking, but review recommended):")
    for chunk in response.chunks:
        if chunk.status == "WARN":
            print(f"  Chunk {chunk.index}: {chunk.message}")
else:
    print("✅ VALIDATION PASSED - No issues found")
print(f"{'='*60}")


In [ ]:
# Display attempt history
if attempts:
    print(f"\nValidation Attempt History ({len(attempts)} attempts):\n")
    for i, attempt in enumerate(attempts, 1):
        print(f"Attempt {i}:")
        print(f"  Timestamp: {attempt['timestamp']}")
        print(f"  Run ID: {attempt['run_id']}")
        print(f"  Status: {attempt['pipeline_status']}")
        s = attempt['summary']
        total = s['pass'] + s['fail'] + s['warn'] + s['error']
        print(f"  Summary: PASS={s['pass']}/{total}, FAIL={s['fail']}/{total}, WARN={s['warn']}/{total}")
        print()
else:
    print("No validation attempts yet. Run the validation cell above.")
